# Metrics extraction

In [1]:
import pandas as pd
import numpy as np

In [2]:
class DilithiumStats:
    def __init__(self, design_id: str, initial_tv: int, last_tv: int):
        self.base_template_vars = {
            "design_id": design_id,
            "initial_tv": initial_tv,
            "last_tv": last_tv
        }
        self.file_template = "{op_type}_{design_id}_lvl{sec_level}_tv{initial_tv}_{last_tv}.csv"

    def get_filename(self, op_type, sec_level):
        return self.file_template.format(op_type=op_type, sec_level=sec_level, **self.base_template_vars)

    def get_keygen_data(self):
        keygen_lvl2_df = pd.read_csv(self.get_filename(op_type="keygen", sec_level="2")).drop(labels="success", axis=1)
        keygen_lvl3_df = pd.read_csv(self.get_filename(op_type="keygen", sec_level="3")).drop(labels="success", axis=1)
        keygen_lvl5_df = pd.read_csv(self.get_filename(op_type="keygen", sec_level="5")).drop(labels="success", axis=1)
        keygen_lvl5_df = keygen_lvl5_df.rename(columns={col: f"{col}_lvl5" for col in keygen_lvl5_df.columns if "cycles" in col})
        keygen_df = pd.merge(keygen_lvl2_df, keygen_lvl3_df, on='test_num', suffixes=["_lvl2", "_lvl3"])
        keygen_df = pd.merge(keygen_df, keygen_lvl5_df, on='test_num', suffixes=[None, "_lvl5"])
        return keygen_df
    
    def get_sign_data(self):
        sign_lvl2_df = pd.read_csv(self.get_filename(op_type="sign", sec_level="2")).drop(labels="success", axis=1)
        sign_lvl3_df = pd.read_csv(self.get_filename(op_type="sign", sec_level="3")).drop(labels="success", axis=1)
        sign_lvl5_df = pd.read_csv(self.get_filename(op_type="sign", sec_level="5")).drop(labels="success", axis=1)

        for current_sign_df in [sign_lvl2_df, sign_lvl3_df, sign_lvl5_df]:
            current_sign_df['rejects_tries_count'] = (current_sign_df['rejects_count'] + 1)
            current_sign_df.drop(labels="rejects_count", axis=1, inplace=True)
        sign_lvl5_df = sign_lvl5_df.rename(columns={col: f"{col}_lvl5" for col in sign_lvl5_df.columns if (("cycles" in col) or ("tries" in col))})

        sign_df = pd.merge(sign_lvl2_df, sign_lvl3_df, on='test_num', suffixes=["_lvl2", "_lvl3"])
        sign_df = pd.merge(sign_df, sign_lvl5_df, on='test_num', suffixes=[None, "_lvl5"])
        return sign_df
    
    def get_verify_data(self):
        verify_lvl2_df = pd.read_csv(self.get_filename(op_type="verify", sec_level="2")).drop(labels="success", axis=1)
        verify_lvl3_df = pd.read_csv(self.get_filename(op_type="verify", sec_level="3")).drop(labels="success", axis=1)
        verify_lvl5_df = pd.read_csv(self.get_filename(op_type="verify", sec_level="5")).drop(labels="success", axis=1)
        verify_lvl5_df = verify_lvl5_df.rename(columns={col: f"{col}_lvl5" for col in verify_lvl5_df.columns if "cycles" in col})
        verify_df = pd.merge(verify_lvl2_df, verify_lvl3_df, on='test_num', suffixes=["_lvl2", "_lvl3"])
        verify_df = pd.merge(verify_df, verify_lvl5_df, on='test_num', suffixes=[None, "_lvl5"])
        return verify_df

In [3]:
lr_stats = DilithiumStats(design_id="perf0", initial_tv=0, last_tv=9)
hp_stats = DilithiumStats(design_id="perf1", initial_tv=0, last_tv=9)

## Key generation

In [4]:
hp_keygen_df = hp_stats.get_keygen_data()
lr_keygen_df = lr_stats.get_keygen_data()

In [5]:
hp_keygen_df.describe()

,test_num,total_cycles_lvl2,total_cycles_lvl3,total_cycles_lvl5
count,10.00000,10.000000,10.000000,10.000000
mean,4.50000,4868.300000,8289.300000,14030.800000
std,3.02765,3.267687,2.110819,5.884065
min,0.00000,4863.000000,8286.000000,14022.000000
25%,2.25000,4866.000000,8288.000000,14028.000000
50%,4.50000,4868.500000,8289.500000,14028.000000
75%,6.75000,4870.750000,8290.000000,14034.250000
max,9.00000,4873.000000,8294.000000,14040.000000


In [6]:
lr_keygen_df.describe()

,test_num,load_cycles_lvl2,exec_cycles_lvl2,unload_cycles_lvl2,total_cycles_lvl2,load_cycles_lvl3,exec_cycles_lvl3,unload_cycles_lvl3,total_cycles_lvl3,load_cycles_lvl5,exec_cycles_lvl5,unload_cycles_lvl5,total_cycles_lvl5
count,10.00000,10.0,10.000000,10.0,10.000000,10.0,10.000000,10.0,10.000000,10.0,10.000000,10.0,10.000000
mean,4.50000,9.0,18724.600000,2882.0,21615.600000,9.0,33045.300000,5586.0,38640.300000,9.0,50987.600000,5602.0,56598.600000
std,3.02765,0.0,56.042048,0.0,56.042048,0.0,59.355895,0.0,59.355895,0.0,94.023874,0.0,94.023874
min,0.00000,9.0,18646.000000,2882.0,21537.000000,9.0,32966.000000,5586.0,38561.000000,9.0,50837.000000,5602.0,56448.000000
25%,2.25000,9.0,18682.000000,2882.0,21573.000000,9.0,32986.750000,5586.0,38581.750000,9.0,50918.000000,5602.0,56529.000000
50%,4.50000,9.0,18721.000000,2882.0,21612.000000,9.0,33061.500000,5586.0,38656.500000,9.0,50993.500000,5602.0,56604.500000
75%,6.75000,9.0,18751.000000,2882.0,21642.000000,9.0,33090.250000,5586.0,38685.250000,9.0,51030.000000,5602.0,56641.000000
max,9.00000,9.0,18813.000000,2882.0,21704.000000,9.0,33132.000000,5586.0,38727.000000,9.0,51163.000000,5602.0,56774.000000


## Verify

In [7]:
hp_verify_df = hp_stats.get_verify_data()
lr_verify_df = lr_stats.get_verify_data()

In [8]:
hp_verify_df.describe()

,test_num,total_cycles_lvl2,total_cycles_lvl3,total_cycles_lvl5
count,10.00000,10.0,10.0,10.000000
mean,4.50000,6581.0,9723.0,14636.800000
std,3.02765,0.0,0.0,5.884065
min,0.00000,6581.0,9723.0,14628.000000
25%,2.25000,6581.0,9723.0,14634.000000
50%,4.50000,6581.0,9723.0,14634.000000
75%,6.75000,6581.0,9723.0,14640.250000
max,9.00000,6581.0,9723.0,14646.000000


In [9]:
lr_verify_df.describe()

,test_num,load_pk_cycles_lvl2,load_sig_cycles_lvl2,load_msg_cycles_lvl2,exec_cycles_lvl2,total_cycles_lvl2,load_pk_cycles_lvl3,load_sig_cycles_lvl3,load_msg_cycles_lvl3,exec_cycles_lvl3,total_cycles_lvl3,load_pk_cycles_lvl5,load_sig_cycles_lvl5,load_msg_cycles_lvl5,exec_cycles_lvl5,total_cycles_lvl5
count,10.00000,10.0,10.000000,10.000000,10.000000,10.000000,10.0,10.000000,10.000000,10.00000,10.000000,10.0,10.000000,10.000000,10.000000,10.000000
mean,4.50000,581.0,11863.300000,87.100000,8833.500000,21364.900000,869.0,21191.600000,87.100000,12147.80000,34295.500000,1157.0,37962.200000,87.100000,16522.700000,55729.000000
std,3.02765,0.0,5.831905,47.899084,9.834746,48.886376,0.0,4.765618,47.899084,10.87096,49.148867,0.0,15.017027,47.899084,8.353974,47.879478
min,0.00000,581.0,11851.000000,22.000000,8817.000000,21308.000000,869.0,21183.000000,22.000000,12135.00000,34244.000000,1157.0,37941.000000,22.000000,16510.000000,55670.000000
25%,2.25000,581.0,11860.000000,53.000000,8827.500000,21333.750000,869.0,21190.000000,53.000000,12141.25000,34267.000000,1157.0,37953.750000,53.000000,16518.500000,55703.750000
50%,4.50000,581.0,11863.500000,84.000000,8833.500000,21362.000000,869.0,21191.000000,84.000000,12145.00000,34280.500000,1157.0,37957.000000,84.000000,16520.500000,55714.000000
75%,6.75000,581.0,11867.750000,114.000000,8838.500000,21383.500000,869.0,21194.000000,114.000000,12152.25000,34310.000000,1157.0,37972.000000,114.000000,16526.250000,55750.250000
max,9.00000,581.0,11871.000000,179.000000,8848.000000,21475.000000,869.0,21200.000000,179.000000,12173.00000,34411.000000,1157.0,37990.000000,179.000000,16541.000000,55833.000000


## Signing

In [11]:
hp_sign_df = hp_stats.get_sign_data()
lr_sign_df = lr_stats.get_sign_data()

In [12]:
hp_sign_df.describe()

,test_num,total_cycles_lvl2,rejects_tries_count_lvl2,total_cycles_lvl3,rejects_tries_count_lvl3,total_cycles_lvl5,rejects_tries_count_lvl5
count,10.00000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,4.50000,29012.20000,4.100000,51048.700000,5.300000,93435.800000,7.200000
std,3.02765,22547.02134,3.871549,43925.966423,5.417051,76419.093331,6.908931
min,0.00000,10945.00000,1.000000,16177.000000,1.000000,24846.000000,1.000000
25%,2.25000,12437.50000,1.250000,18209.000000,1.250000,38686.250000,2.250000
50%,4.50000,22582.50000,3.000000,28347.000000,2.500000,69116.000000,5.000000
75%,6.75000,35719.75000,5.250000,89164.500000,10.000000,129939.750000,10.500000
max,9.00000,80847.00000,13.000000,121594.000000,14.000000,268203.000000,23.000000


### Best time

In [17]:
def calculate_best_and_mean_time(dataframe, cols_names_list):
    result_dict = dict()

    for sec_lvl in [2, 3, 5]:
        result_dict[sec_lvl] = dict()
        for col_name in cols_names_list:
            best_time = dataframe[dataframe[f'rejects_tries_count_lvl{sec_lvl}'] == 1][f'{col_name}_lvl{sec_lvl}'].mean()
            mean_time = dataframe[f'{col_name}_lvl{sec_lvl}'].mean()
            result_dict[sec_lvl][col_name] = {'best': best_time, 'mean': mean_time}

    return result_dict

In [19]:
calculate_best_and_mean_time(hp_sign_df, ['total_cycles'])

{2: {'total_cycles': {'best': np.float64(10966.666666666666),
   'mean': np.float64(29012.2)}},
 3: {'total_cycles': {'best': np.float64(16178.666666666666),
   'mean': np.float64(51048.7)}},
 5: {'total_cycles': {'best': np.float64(24853.0),
   'mean': np.float64(93435.8)}}}

In [16]:
lr_sign_df.columns

Index(['test_num', 'load_sk_cycles_lvl2', 'load_msg_cycles_lvl2',
       'exec_cycles_lvl2', 'unload_cycles_lvl2', 'total_cycles_lvl2',
       'rejects_tries_count_lvl2', 'load_sk_cycles_lvl3',
       'load_msg_cycles_lvl3', 'exec_cycles_lvl3', 'unload_cycles_lvl3',
       'total_cycles_lvl3', 'rejects_tries_count_lvl3', 'load_sk_cycles_lvl5',
       'load_msg_cycles_lvl5', 'exec_cycles_lvl5', 'unload_cycles_lvl5',
       'total_cycles_lvl5', 'rejects_tries_count_lvl5'],
      dtype='object')

In [ ]:
calculate_best_and_mean_time(
    lr_sign_df,
    ['load_sk_cycles', 'load_msg_cycles', 'exec_cycles', 'unload_cycles', 'total_cycles']
)

{2: {'total_cycles': nan},
 3: {'total_cycles': np.float64(49818.0)},
 5: {'total_cycles': np.float64(76754.33333333333)}}

### Reject loop time estimation

In [17]:
def estimate_reject_loop_time(dataframe, cycles_col_name: str):
    result_list = []

    for sec_lvl in [2, 3, 5]:
        estimatives = []
        tries_col_name_lvl = f'rejects_tries_count_lvl{sec_lvl}'
        cycles_col_name_lvl = f'{cycles_col_name}_lvl{sec_lvl}'

        retries_num_list = sorted(list(dataframe[tries_col_name_lvl].unique()))
        for idx in range(1, len(retries_num_list)):
            current_retry_num = retries_num_list[idx]
            previous_retry_num = retries_num_list[idx-1]
            retry_num_delta = current_retry_num - previous_retry_num

            current_time = dataframe[dataframe[tries_col_name_lvl] == current_retry_num][cycles_col_name_lvl].mean()
            previous_time = dataframe[dataframe[tries_col_name_lvl] == previous_retry_num][cycles_col_name_lvl].mean()
            estimatives.append((current_time-previous_time)/retry_num_delta)

        result_list.append(np.mean(estimatives))

    return tuple(result_list)

In [ ]:
estimate_reject_loop_time(dataframe=hp_sign_df, cycles_col_name="total_cycles")